In [1]:
import os, sys
sys.path.append( '..' )

In [2]:
import argparse
import pickle as pkl
import pandas as pd
import numpy as np
from collections import Counter
#from wsi_data import subset_wsi_dataset

import torch
from torch import nn, optim
import torch.multiprocessing as mp
from torch.utils.data import Subset

import copy as cp
import yaml
from sklearn.model_selection import KFold, StratifiedKFold

import matplotlib.pyplot as plt

In [3]:
from model import HE2Omics, fit, predict
from experiment import Experiment
from copy import deepcopy

In [4]:
from wsi_data import patient_stratified_kfold
#from image_materials.model import ImageToSubtype

#from image_materials.experiment import ImageSetup
#from image_materials.train import *

In [5]:
rootdir = '/gstore/data/tmai/Yasin/TMONC/HE2OMICS/TRL/2024_1_8'
savedir = './FOLDS/'
logdir = './exp'

In [6]:
def subset_wsi_dataset(dataset, subset_ix):
    """ Subset WSI dataset class instance with indices from subset_ix
    """
    dataset_copy = deepcopy(dataset)
    dataset_copy.samples = [dataset.samples[ix] for ix in subset_ix]
    dataset_copy.patients = np.array([dataset.patients[ix] for ix in subset_ix])
    dataset_copy.trials = pd.Series([dataset.trials[ix] for ix in subset_ix])
    dataset_copy.subtypes = pd.Series([dataset.subtypes[ix] for ix in subset_ix])
    return(dataset_copy)  

In [7]:
def convert_to_subtype_prediction(dataset,class_names,scaleFeatures=True,n_tiles=False):
    """
    Change (image,expression) tuples to (image,subtype), and make AlltilesWSI dataset
    """
    # string names for subtypes
    subtype = [str(i) for i in list(group2_dataset.subtypes)]
    # convert to indices
    subtype = [class_names.index(name) for name in subtype]
    
    dataset_copy = deepcopy(dataset)
    new_samples = [(image,int(subtype[idx])) for idx,(image,_) in enumerate(dataset_copy.samples)]
    #dataset_copy.samples = new_samples
    subtype_dataset = AlltilesWSI(new_samples,scaleFeatures=scaleFeatures,n_tiles=n_tiles)
    return(subtype_dataset)

### Define config files, use 50 most discriminant genes in HE2Omics

In [8]:
confi = yaml.load(open(os.path.join(rootdir,"config_main_50genes.yaml"), "r"), Loader=yaml.FullLoader)
print('Using configuration defined in {}'.format(confi['config']))

Using configuration defined in /gstore/data/tmai/Yasin/TMONC/HE2OMICS/TRL/2024_1_8/configs/IMvigor_3trial_50genes.ini


### Build dataset and permute patients stratifying on trial and subtype

In [9]:
config = confi['config']
exp = Experiment(config)

# Read architecture and training parameters
model_params = exp._read_architecture()
training_params = exp._read_training_params()

# Build dataset
dataset = exp._build_dataset()
# UNIQUE PATIENT IDENTIFIERS
pat_ids_unique, uniq_indices = np.unique(dataset.patients,return_index=True)
TOT_pat = pat_ids_unique.shape[0]
print(f'Number of patients: {TOT_pat}')

# STRATIFY BY TRIAL AND SUBTYPE
class_list = []
for trial,subtype in zip(dataset.trials,dataset.subtypes):
    class_list.append(trial+'_NMF'+str(subtype))
class_list = np.array(class_list)
class_list_patients = class_list[uniq_indices]

# PARAMETERS
Nsplit = 10
kfold_group1 = 5

# PERMUTE PATIENTS
ind_testing = np.zeros((Nsplit, TOT_pat))
np.random.seed(42)
for i in range(Nsplit):
    ind_testing[i, :] = np.random.permutation(TOT_pat)
ind_testing = ind_testing.astype(int)

logTransform: False, scaleVariables: False, ensureNonnegativity: True
Number of variables in input: 51
Number of genes found in transcriptome data: 51
Filtered out 161 slides less than 1000 tiles
number of images with paired omic data: 2285
Number of patients: 1810


In [ ]:
# SEPARATE GROUP1 AND GROUP2 PATIENTS
# Group 1 gets the patients for transcriptomic learning
# Group 2 is the patient set where the image-based model will be compared with transfer learning (the transcriptomic model from group 1)

start_split = 2
start_fold = 1

# loop from 0 to Nsplit
for rs in range(start_split,Nsplit):
    ind = ind_testing[rs, :]
    shuffled_patient_classes = class_list_patients[ind]
    shuffled_patients = pat_ids_unique[ind]

    skf = StratifiedKFold(n_splits=2, shuffle=False) # shuffling is done earlier with permutations

    # Transcriptomic learning gets n_splits-1 folds (index1), classifier training gets 1 fold (index 2)
    # index1, index2 = next(iter(skf.split(ind, shuffled_patient_classes)))
    group1_index, group2_index = next(iter(skf.split(ind, shuffled_patient_classes)))

    print(f'Number of group 1 patients: {len(group1_index)}, Number of group 2 patients: {len(group2_index)}')

    pat_G1 = shuffled_patients[group1_index]
    pat_G2 = shuffled_patients[group2_index]
    pat_G1_classes = shuffled_patient_classes[group1_index]
    pat_G2_classes = shuffled_patient_classes[group2_index]
    
    # Make group 1 and group 2 datasets
    slide_ix_group1 = [ix for ix, d in enumerate(dataset.patients) if d in pat_G1]
    slide_ix_group2 = [ix for ix, d in enumerate(dataset.patients) if d in pat_G2]
    group1_dataset = subset_wsi_dataset(dataset,slide_ix_group1)
    group2_dataset = subset_wsi_dataset(dataset,slide_ix_group2)
    
    print(f'Number of group 1 images: {len(group1_dataset)}, Number of group 2 images: {len(group2_dataset)}')
    
    # Stratified KFold to train the gene prediction model within group 1 slides
    # Cross-validation will be done over slides (not patients) because inference will be on group 2 slides
    skf = StratifiedKFold(n_splits=kfold_group1, shuffle=True, random_state=1)

    # Stratify by TRIAL and SUBTYPE
    g1_class_list = []
    for trial,subtype in zip(group1_dataset.trials,group1_dataset.subtypes):
        g1_class_list.append(trial+'_NMF'+str(subtype))
    g1_class_list = np.array(g1_class_list)
    
    # Creating a folder for each split
    model_savedir = os.path.join(exp.savedir,'split'+str(rs))
    os.makedirs(model_savedir, exist_ok=True)
        
    # Run cross-validation
    for fold, (train_index, valid_index) in enumerate(skf.split(np.zeros((len(group1_dataset), 1)), g1_class_list)):

        if rs==start_split and fold < start_fold:
            continue
        print(f'Running split {str(rs)}, fold {str(fold)}')

        # If pickle file for samples does not exist, define and save it
        os.makedirs(savedir, exist_ok=True)
        path_pickle_file = os.path.join(savedir,'train_and_valid_set_ids_fold'+ str(fold) + '.pkl')
        if not os.path.exists(path_pickle_file):
            train_he = []
            for idx in train_index:
                filename = os.path.basename(group1_dataset.samples[idx][0]).replace('.npy','')
                train_he.append(filename)

            valid_he = []
            for idx in valid_index:
                filename = os.path.basename(group1_dataset.samples[idx][0]).replace('.npy','')
                valid_he.append(filename)

            # save train and validation slide identifers
            he_tuple = (train_he,valid_he)
            with open(path_pickle_file, 'wb') as handle:
                pkl.dump(he_tuple, handle)

        # subset train and validation sets
        train_set = Subset(group1_dataset, train_index)
        valid_set = Subset(group1_dataset, valid_index)
        valid_ctype_integers = np.zeros((len(valid_index))) # all zeroes since all samples are from the same cancer type
        model_params['input_dim'] = group1_dataset.dim # 2048
        model_params['output_dim'] = len(dataset.variables)
        print('Features starting with: {}'.format(dataset.variables[0][:4]))

        # Initialize bias of the last layer with the average target value on the train set
        try:
            model_params['bias_init'] = torch.nn.Parameter(torch.Tensor(np.mean([sample[1] for sample in train_set], axis=0)).cuda())
        except ValueError:
            model_params['bias_init'] = torch.nn.Parameter(torch.Tensor(np.mean([sample[1].numpy() for sample in train_set], axis=0)).cuda())

        model = HE2Omics(**model_params)
        optimizer = exp._setup_optimization(model)
        scheduler = optim.lr_scheduler.StepLR(optimizer,step_size=5,gamma=0.9)
        
        print("about to train model")
        test_preds, test_labels, valid_preds, valid_labels = fit(model,
                            train_set,
                            valid_set,
                            valid_ctype_integers,
                            params=training_params,
                            optimizer=optimizer,
                            scheduler=scheduler,
                            logdir=logdir,
                            path=model_savedir,
                            fold=fold)

Number of group 1 patients: 905, Number of group 2 patients: 905
Number of group 1 images: 1159, Number of group 2 images: 1126
Running split 2, fold 1
Features starting with: GENE
about to train model
Mean spearman r: 0.026


  0%|          | 0/29 [00:00<?, ?it/s]

Num epochs since best: 1


100%|██████████| 29/29 [01:11<00:00,  2.46s/it]


Epoch 1/300 - 119.76s
loss: 0.9922, val loss: 1.0465
Mean spearman r: 0.238


...

100%|██████████| 29/29 [01:20<00:00,  2.77s/it]


Epoch 54/300 - 107.52s


  0%|          | 0/29 [00:00<?, ?it/s]

loss: 0.4554, val loss: 0.6664
Mean spearman r: 0.579
Num epochs since best: 11


 86%|████████▌ | 25/29 [01:15<00:11,  2.99s/it]